In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pytorch_lightning as pl
from huggingface_hub import notebook_login
from omegaconf import OmegaConf

from src import lightning_model
from src.data import get_dataloaders

# Fix seed
pl.seed_everything(0, workers=True)

config = OmegaConf.load('config.yaml')

Seed set to 0


In [2]:
train_loader, val_loader = get_dataloaders(**config.data)

22395it [00:00, 426854.37it/s]


Loading formatted geocoded file...
Dataset size: 22395


In [3]:
model = lightning_model.BaseModel(config)

In [4]:
trainer = pl.Trainer(max_epochs=config.train.epochs,
                    enable_checkpointing=True,
                    callbacks=[lightning_model.OverrideEpochStepCallback(),
                                pl.callbacks.EarlyStopping(monitor='val/loss', patience=config.train.patience, verbose=False),
                                pl.callbacks.ModelCheckpoint(monitor='val/loss', filename='best_model_{epoch:02d}', mode='min')
                                ]
                    )
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                          | Params
----------------------------------------------------------------------
0 | img_encoder         | CLIPVisionModelWithProjection | 38.6 M
1 | location_identifier | Sequential                    | 789 K 
----------------------------------------------------------------------
39.4 M    Trainable params
0         Non-trainable params
39.4 M    Total params
157.507   Total estimat

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/yosshstd/anaconda3/envs/deeplearning/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


/home/yosshstd/anaconda3/envs/deeplearning/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 128. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 0: 100%|██████████| 331/331 [05:05<00:00,  1.08it/s, v_num=19, train/loss_step=1.94e+3]

...

Epoch 0: 100%|██████████| 331/331 [05:05<00:00,  1.08it/s, v_num=19, train/loss_step=757.0]  

/home/yosshstd/anaconda3/envs/deeplearning/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 95. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 49: 100%|██████████| 331/331 [05:26<00:00,  1.01it/s, v_num=19, train/loss_step=70.20, val/loss_step=734.0, val/loss_epoch=903.0, train/loss_epoch=90.70]       

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 331/331 [05:26<00:00,  1.01it/s, v_num=19, train/loss_step=70.20, val/loss_step=734.0, val/loss_epoch=903.0, train/loss_epoch=90.70]


In [5]:
%load_ext tensorboard
%tensorboard  --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 12332), started 3 days, 4:10:56 ago. (Use '!kill 12332' to kill it.)

In [54]:
# # load best model
# model = lightning_model.BaseModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path, config=config)

In [111]:
# # Save model
# os.makedirs('models', exist_ok=True)
# torch.save(model.state_dict(), f'models/{config.data.model_name.split("/")[-1]}_location_identifier_best.pth')

In [4]:
image, true_coord, txt_country_embeds, country_name = next(iter(val_loader))

In [12]:
id = np.random.randint(0, 128)

# Forward pass
model.eval()
with torch.no_grad():
    pred_coord = model(image.to(model.device))

print(f'True coord: {true_coord[id]}')
print(f'Pred coord: {pred_coord[id]}')
print(f'True country: {country_name[id]}')

print(f'shape: {txt_country_embeds.shape}')

True coord: tensor([-12.0013, -77.1297])
Pred coord: tensor([-13.1685, -72.3047])
True country: A street view photo in Callao, Callao, Peru.
shape: torch.Size([128, 512])


In [9]:
import math

def haversine(lat1, lon1, lat2, lon2):
    """
    2点間の距離を計算する関数(haversine公式)
    引数:
        lat1, lon1: 点1の緯度と経度(度)
        lat2, lon2: 点2の緯度と経度(度)
    返り値:
        2点間の距離(km)
    """
    
    # 度からラジアンに変換
    lat1 = lat1 * math.pi / 180
    lon1 = lon1 * math.pi / 180
    lat2 = lat2 * math.pi / 180 
    lon2 = lon2 * math.pi / 180

    # ハベルサイン公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    # 地球の半径を6371km として距離を計算
    distance = 6371 * c
    
    return distance



In [117]:
# 入力値
true_lat, true_lon = true_coord[id].cpu().numpy()
pred_lat, pred_lon = pred_coord[id].cpu().numpy()

# 距離を計算
distance = haversine(true_lat, true_lon, pred_lat, pred_lon)
print(f"Distance: {distance:.2f} km")

Distance: 6460.40 km


In [168]:
# from huggingface_hub import HfApi
# from huggingface_hub import notebook_login

# notebook_login()
# # Authenticate with your Hugging Face Hub token
# api = HfApi()
# username = api.whoami()["name"]

# model_id = "GeoGuessrAI"
# # Create a repository on the Hugging Face Hub
# repo_url = api.create_repo(repo_id=model_id, exist_ok=True)

In [8]:
# # load model.pth from hugging face
# model_path = 'https://huggingface.co/yosshstd/GeoGuessrAI/resolve/main/model.pth'

# model = lightning_model.BaseModel(config)

# # Hugging Face URL から状態辞書をロードします。(.cacheに保存されます。)
# state_dict = torch.hub.load_state_dict_from_url(model_path, map_location=torch.device('cuda'))

# # ロードした状態辞書をモデルに適用します。
# model.load_state_dict(state_dict)

<All keys matched successfully>